<a href="https://colab.research.google.com/github/subjekt-iv/StyleGAN2-ModernArt/blob/main/Copia_de_stylegan2_multi_pkl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Tue May 17 00:31:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%tensorflow_version 1.x

!git clone https://github.com/dvschultz/stylegan2
!pip install opensimplex # needed for noise interpolation
%cd stylegan2
%mkdir datasets

TensorFlow 1.x selected.
Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/stylegan2


#Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). I recommend doing this on your server because the files become quite large and will be slow to upload over FTP.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [3]:
!python dataset_tool.py create_from_images ./datasets/my-custom-dataset ./my-custom-images

Loading images from "./my-custom-images"
Error: No input images found


#Test the model
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [4]:
!pip install -U --no-cache-dir gdown --pre


In [ ]:
!gdown --id 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1H-MYFZqngF1R0whm4bc3fEoX7VvOWaDl
To: /content/stylegan2/network-snapshot-metfaces2.pkl
100% 382M/382M [00:10<00:00, 35.7MB/s]


In [5]:
!wget  https://archive.org/download/wikiart-stylegan2-conditional-model/network-snapshot-012052.pkl

--2022-05-17 00:35:12--  https://archive.org/download/wikiart-stylegan2-conditional-model/network-snapshot-012052.pkl
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802803.us.archive.org/30/items/wikiart-stylegan2-conditional-model/network-snapshot-012052.pkl [following]
--2022-05-17 00:35:12--  https://ia802803.us.archive.org/30/items/wikiart-stylegan2-conditional-model/network-snapshot-012052.pkl
Resolving ia802803.us.archive.org (ia802803.us.archive.org)... 207.241.232.113
Connecting to ia802803.us.archive.org (ia802803.us.archive.org)|207.241.232.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 384781547 (367M) [application/octet-stream]
Saving to: ‘network-snapshot-012052.pkl’

network-snapshot-01 100%[===================>] 366.96M   433KB/s    in 11m 54s 

2022-05-17 00:47:06 (526 KB/s) - ‘network-snapsho

In [ ]:
!python run_generator.py generate-images --network=/content/stylegan2/stylegan2-ffhq-config-f.pkl --seeds=200000-200050 --truncation-psi=0.6

Let’s zip the generated files and download them.

In [ ]:
!zip -r mundanetoaster2.zip /content/stylegan2/results/00001-generate-images

#Interpolation


In [ ]:
!python run_generator.py generate-latent-walk --network=/content/stylegan2/network-snapshot-metfaces2.pkl --seeds=1,567,334000,33 --frames 400 --truncation-psi=2.0

In [ ]:
#convert to video 
!ffmpeg -r 20 -i ./results/00003-generate-latent-walk/%*.png  -vcodec libx264 -pix_fmt yuv420p output.mp4